## Lab 1 -- Comprehend Lab
Text Classification can be used to solve various use-cases like sentiment analysis, spam detection, hashtag prediction etc. This notebook demonstrates the use of SageMaker BlazingText to perform supervised binary/multi class with single or multi label text classification.

In [ ]:
# IMPORT DEPENDENCIES
import sagemaker
from sagemaker import get_execution_role
import json
import boto3
import botocore
import os
import re
import logging
import time


In [ ]:
#Prepare for the lab by learning the current IAM Role and the S3 bucket we will use...

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

# REPLACE THE POD BELOW WITH THE NUMBER ASSIGN TO YOU 
pod = '9999'
region = 'us-east-2'
prefix = 'comprehend' #we will store the dataset we will use to train a custom classification classifier

bucket = "rga-aws-ai-workshop-pod-" + pod # Replace with your own bucket name if needed
print("The S3 bucket you are using to upload your dataset is:", bucket)

In [ ]:
# Run the first Airbnb review through Comprehend native service.
client = boto3.client('comprehend')

#Lets first run the first Airbnb review through the native sentiment analysis of Comprehend to see whats available
#without customization. Please note this is a simple API call with no ML training required.
textreview = 'We were In Chicago to see Hamilton and sightsee with our teenaged daughter. We loved this location and neighborhood. It felt safe and there were tons of nearby places to eat or get groceries. The condo itself was extremely clean and was well stocked with necessities. It was spacious and we never felt crowded. Rob was a great host and was very responsive to our questions. It was a plus to have parking available as we never moved our car until we left Chicago. I would definitely rent from Rob again.'
print(textreview + '\n')

response = client.detect_sentiment(
    Text=textreview,
    LanguageCode='en'
)
print(json.dumps(response, indent=4))

#Note the ability to extract the overall sentiment from the reviews. Comprehend can provide: positive, negative,
#neutral, and mixed from the text with no training/tuning
#This can also be run from the Comprehend AWS Console interface


In [ ]:
#Lets also use Comprehend to detect entities and key phrases for our Airbnb review...
response = client.detect_entities(
    Text=textreview,
    LanguageCode='en'
)
print(textreview)
print(json.dumps(response, indent=4) + '\n')


response = client.detect_key_phrases(
    Text=textreview,
    LanguageCode='en'
)
print(textreview)
print(json.dumps(response, indent=4))

#Make note of the different information that can be extracted from the review with no ML model training or tuning....


## Instructions to Update the IAM Role for the Sagemaker Notebook
Need to provide access to:

- The S3 bucket we will create
- Comprehend
- Sagemaker stuff
- Create a new IAM role for Comprehend DataAccessRoleArn & include the S3 bucket that was created.

In [ ]:
#CREATE THE BUCKET to be used for the dataset
def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return print(bucket + ' bucket created successfully!')

create_bucket(bucket, region)

In [ ]:
#Inspect the dataset. We are reviewing only the first two entries.
!ls comprehend/ -la
!head comprehend/airbnb-reviews-training.csv -n 2

# Note that the data has two columns, the first is a custom label for 'great' and 'notgreat' to indicate
# the custom labels that have been applied. This is the format required for the Comprehend custom classifier.
# The first column is the custom label, the second column contains the raw review.


In [ ]:
#UPLOAD THE DATASET TO S3 FROM THE LOCAL SYSTEM FOR COMPREHEND CUSTOM TRAINING
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return print(file_name + ' uploaded successfully!')

upload_file('comprehend/airbnb-reviews-training.csv', bucket)
upload_file('comprehend/airbnb-reviews-holdout.csv', bucket)


## Create IAM Role for Comprehend to read S3 data from your bucket

Create Policy:
{
    "Version": "2012-10-17",
    "Statement": {
        "Effect": "Allow",
        "Action": [
            "s3:GetObject",
            "s3:ListBucket",
            "s3:PutObject"
        ],
        "Resource": [
            "arn:aws:s3:::bucketname"
        ]
    }
}

Create Role:


In [ ]:
# Instantiate Boto3 SDK:
client = boto3.client('comprehend', region_name=region)
s3_uri = 's3://' + bucket + '/' + prefix + '/' + 'airbnb-reviews-training.csv'

#UPDATE THE ARN FROM THE ROLE YOU CREATED IN THE PREVIOUS SECTION
account_id = role.split(':')[4]
dataaccessarn='arn:aws:iam::' + account_id + ':role/comprehend-rga'

docclassifier='RGAAirbnb-' + pod

# Create a document classifier
create_response = client.create_document_classifier(
    InputDataConfig={
        'S3Uri': s3_uri
    },
    DataAccessRoleArn=dataaccessarn,
    DocumentClassifierName=docclassifier,
    LanguageCode='en'
)
print('Create response:\n' + str(create_response))

# Check the status of the classifier
docclassifierarn = create_response['DocumentClassifierArn']
describe_response = client.describe_document_classifier(
    DocumentClassifierArn=docclassifierarn)
print('\nDescribe response:\n' + str(describe_response))


In [ ]:
# Check the status of the classifier
# Look for a status on TRAINED before moving to the next step
describe_response = client.describe_document_classifier(DocumentClassifierArn=docclassifierarn)
print('Document classifier status:')
print(describe_response['DocumentClassifierProperties']['Status'])
while describe_response['DocumentClassifierProperties']['Status'] != 'TRAINED':
    time.sleep(20)
    describe_response = client.describe_document_classifier(DocumentClassifierArn=docclassifierarn)
    print(describe_response['DocumentClassifierProperties']['Status'])

print('\nCustom Model Accuracy:\n' + str(describe_response['DocumentClassifierProperties']['ClassifierMetadata']['EvaluationMetrics']))

# Alternatively please review the Comprehend AWS Console GUI to validate if your custom job is still training

In [ ]:
client = boto3.client('comprehend', region_name=region)
s3_uri_in = 's3://' + bucket + '/' + prefix + '/' + 'airbnb-reviews-holdout.csv'
s3_uri_out = 's3://' + bucket + '/' + prefix + '/' + 'output'
jobname='RGAAirbnb-Job-' + pod

start_response = client.start_document_classification_job(
    InputDataConfig={
        'S3Uri': s3_uri_in,
        'InputFormat': 'ONE_DOC_PER_LINE'
    },
    OutputDataConfig={
        'S3Uri': s3_uri_out
    },
    DataAccessRoleArn=dataaccessarn,
    DocumentClassifierArn=docclassifierarn,
    JobName=jobname
)

print("Start response:\n", start_response)

# Check the status of the job
describe_response = client.describe_document_classification_job(JobId=start_response['JobId'])
print("\nDescribe response:\n", describe_response)


In [ ]:
# Check the status of the job.
# When JobStatus is COMPLETED you can move to the next step.
describe_response = client.describe_document_classification_job(JobId=start_response['JobId'])
print('Job training status:')
print(describe_response['DocumentClassificationJobProperties']['JobStatus'])
while describe_response['DocumentClassificationJobProperties']['JobStatus'] != 'COMPLETED':
    time.sleep(10)
    describe_response = client.describe_document_classification_job(JobId=start_response['JobId'])
    print(describe_response['DocumentClassificationJobProperties']['JobStatus'])

In [ ]:
output_s3 = describe_response['DocumentClassificationJobProperties']['OutputDataConfig']['S3Uri']
#print(output_s3)
!aws s3 cp {output_s3} comprehend/output.tar.gz
!tar -xvzf comprehend/output.tar.gz -C comprehend
!head comprehend/predictions.jsonl -n 20


## Lab 2 -- Sagemaker model training & delivery using a native algorithm (BlazingText)

Text Classification can be used to solve various use-cases like sentiment analysis, spam detection, hashtag prediction etc. This notebook demonstrates the use of SageMaker BlazingText to perform supervised binary/multi class with single or multi label text classification. BlazingText can train the model on more than a billion words in a couple of minutes using a multi-core CPU or a GPU, while achieving performance on par with the state-of-the-art deep learning text classification algorithms. BlazingText extends the fastText text classifier to leverage GPU acceleration using custom CUDA kernels.

## Setup

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region. 
- The IAM role ARN used to give SageMaker access to your data. It can be fetched using the **get_execution_role** method from sagemaker python SDK.

In [61]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3
import botocore
import os
import re

sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

pod = '9999'
region = 'us-east-2'
prefix = 'blazingtext' #we will store the dataset we will use to train a custom classification classifier

bucket = "rga-aws-ai-workshop-pod-" + pod # Replace with your own bucket name if needed
print(bucket)

arn:aws:iam::969219788367:role/service-role/AmazonSageMaker-ExecutionRole-20181112T160917
rga-aws-ai-workshop-pod-9999


### Data Preparation

Now we'll download a dataset from the web on which we want to train the text classification model. BlazingText expects a single preprocessed text file with space separated tokens and each line of the file should contain a single sentence and the corresponding label(s) prefixed by "\__label\__".

In this example, let us train the text classification model on the [DBPedia Ontology Dataset](https://wiki.dbpedia.org/services-resources/dbpedia-data-set-2014#2) as done by [Zhang et al](https://arxiv.org/pdf/1509.01626.pdf). The DBpedia ontology dataset is constructed by picking 14 nonoverlapping classes from DBpedia 2014. It has 560,000 training samples and 70,000 testing samples. The fields we used for this dataset contain title and abstract of each Wikipedia article. 

In [65]:

labels_file = "classes.txt"
train_file = "train.csv"
test_file = "test.csv"

!ls {prefix} -la


total 820
drwxrwxr-x 2 ec2-user ec2-user   4096 Oct 16 01:16 .
drwxrwxrwx 5 ec2-user ec2-user   4096 Oct 16 01:16 ..
-rw-rw-r-- 1 ec2-user ec2-user     18 Oct 16 01:16 classes.txt
-rw-rw-r-- 1 ec2-user ec2-user  22881 Oct 16 01:16 test.csv
-rw-rw-r-- 1 ec2-user ec2-user 799616 Oct 16 01:16 train.csv


Let us inspect the dataset and the classes to get some understanding about how the data and the label is provided in the dataset. 

In [67]:
!head {prefix}/train.csv -n 3

As can be seen from the above output, the CSV has 3 fields - Label index, title and abstract. Let us first create a label index to label name mapping and then proceed to preprocess the dataset for ingestion by BlazingText.

Next we will print the labels file (`classes.txt`) to see all possible labels followed by creating an index to label mapping.

In [68]:
!cat {prefix}/classes.txt



Great

The following code creates the mapping from integer indices to class label which will later be used to retrieve the actual class name during inference. 

In [69]:
#index_to_label = {} 
#with open("dbpedia_csv/classes.txt") as f:
#    for i,label in enumerate(f.readlines()):
#        index_to_label[str(i+1)] = label.strip()
#print(index_to_label)

index_to_label = {} 
with open("classes.txt", mode='r', encoding='utf-8-sig') as f:
    for i,label in enumerate(f.readlines()):
        index_to_label[str(i)] = label.strip()
print(index_to_label)

{'0': 'NotGreat', '1': 'Great'}


## Data Preprocessing
We need to preprocess the training data into **space separated tokenized text** format which can be consumed by `BlazingText` algorithm. Also, as mentioned previously, the class label(s) should be prefixed with `__label__` and it should be present in the same line along with the original sentence. We'll use `nltk` library to tokenize the input sentences from DBPedia dataset. 

Download the nltk tokenizer and other libraries

In [70]:
from random import shuffle
import multiprocessing
from multiprocessing import Pool
import csv
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [71]:
def transform_instance(row):
    cur_row = []
    label = "__label__" + index_to_label[row[0]]  #Prefix the index-ed label with __label__
    cur_row.append(label)
    cur_row.extend(nltk.word_tokenize(row[1].lower()))
    cur_row.extend(nltk.word_tokenize(row[2].lower()))
    return cur_row

The `transform_instance` will be applied to each data instance in parallel using python's multiprocessing module

In [72]:
def preprocess(input_file, output_file, keep=1):
    all_rows = []
    with open(input_file, mode='r', encoding='utf-8-sig') as csvinfile:
        csv_reader = csv.reader(csvinfile, delimiter=',')
        for row in csv_reader:
            all_rows.append(row)
    shuffle(all_rows)
    all_rows = all_rows[:int(keep*len(all_rows))]
    pool = Pool(processes=multiprocessing.cpu_count())
    transformed_rows = pool.map(transform_instance, all_rows)
    pool.close() 
    pool.join()
    
    with open(output_file, 'w') as csvoutfile:
        csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n')
        csv_writer.writerows(transformed_rows)

In [73]:
%%time

# Preparing the training dataset

# Since preprocessing the whole dataset might take a couple of mintutes,
# we keep 20% of the training dataset for this demo.
# Set keep to 1 if you want to use the complete dataset
preprocess('train.csv', 'airbnb.train', keep=1)
        
# Preparing the validation dataset        
preprocess('test.csv', 'airbnb.validation')

CPU times: user 55.1 ms, sys: 45 ms, total: 100 ms
Wall time: 898 ms


The data preprocessing cell might take a minute to run. After the data preprocessing is complete, we need to upload it to S3 so that it can be consumed by SageMaker to execute training jobs. We'll use Python SDK to upload these two files to the bucket and prefix location that we have set above.   

In [74]:
%%time

train_channel = prefix + '/train'
validation_channel = prefix + '/validation'

sess.upload_data(path='airbnb.train', bucket=bucket, key_prefix=train_channel)
sess.upload_data(path='airbnb.validation', bucket=bucket, key_prefix=validation_channel)

s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)

CPU times: user 35.9 ms, sys: 20.1 ms, total: 56 ms
Wall time: 205 ms


Next we need to setup an output location at S3, where the model artifact will be dumped. These artifacts are also the output of the algorithm's traning job.

In [75]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

## Training
Now that we are done with all the setup that is needed, we are ready to train our object detector. To begin, let us create a ``sageMaker.estimator.Estimator`` object. This estimator will launch the training job.

In [76]:
region_name = boto3.Session().region_name

In [77]:
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

Using SageMaker BlazingText container: 825641698319.dkr.ecr.us-east-2.amazonaws.com/blazingtext:latest (us-east-2)


## Training the BlazingText model for supervised text classification

Similar to the original implementation of [Word2Vec](https://arxiv.org/pdf/1301.3781.pdf), SageMaker BlazingText provides an efficient implementation of the continuous bag-of-words (CBOW) and skip-gram architectures using Negative Sampling, on CPUs and additionally on GPU[s]. The GPU implementation uses highly optimized CUDA kernels. To learn more, please refer to [*BlazingText: Scaling and Accelerating Word2Vec using Multiple GPUs*](https://dl.acm.org/citation.cfm?doid=3146347.3146354).




Besides skip-gram and CBOW, SageMaker BlazingText also supports the "Batch Skipgram" mode, which uses efficient mini-batching and matrix-matrix operations ([BLAS Level 3 routines](https://software.intel.com/en-us/mkl-developer-reference-fortran-blas-level-3-routines)). This mode enables distributed word2vec training across multiple CPU nodes, allowing almost linear scale up of word2vec computation to process hundreds of millions of words per second. Please refer to [*Parallelizing Word2Vec in Shared and Distributed Memory*](https://arxiv.org/pdf/1604.04661.pdf) to learn more.

BlazingText also supports a *supervised* mode for text classification. It extends the FastText text classifier to leverage GPU acceleration using custom CUDA kernels. The model can be trained on more than a billion words in a couple of minutes using a multi-core CPU or a GPU, while achieving performance on par with the state-of-the-art deep learning text classification algorithms. For more information, please refer to the [algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html).

To summarize, the following modes are supported by BlazingText on different types instances:

|          Modes         	| cbow (supports subwords training) 	| skipgram (supports subwords training) 	| batch_skipgram 	| supervised |
|:----------------------:	|:----:	|:--------:	|:--------------:	| :--------------:	|
|   Single CPU instance  	|   ✔  	|     ✔    	|        ✔       	|  ✔  |
|   Single GPU instance  	|   ✔  	|     ✔    	|                	|  ✔ (Instance with 1 GPU only)  |
| Multiple CPU instances 	|      	|          	|        ✔       	|     | |

Now, let's define the SageMaker `Estimator` with resource configurations and hyperparameters to train Text Classification on *DBPedia* dataset, using "supervised" mode on a `c4.4xlarge` instance.


In [78]:
bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.c4.xlarge',
                                         train_volume_size = 30,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

Please refer to [algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext_hyperparameters.html) for the complete list of hyperparameters.

In [79]:
bt_model.set_hyperparameters(mode="supervised",
                            epochs=10,
                            min_count=2,
                            learning_rate=0.05,
                            vector_dim=10,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=2)

Now that the hyper-parameters are setup, let us prepare the handshake between our data channels and the algorithm. To do this, we need to create the `sagemaker.session.s3_input` objects from our data channels. These objects are then put in a simple dictionary, which the algorithm consumes.

In [ ]:
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/plain', s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}

We have our `Estimator` object, we have set the hyper-parameters for this object and we have our data channels linked with the algorithm. The only  remaining thing to do is to train the algorithm. The following command will train the algorithm. Training the algorithm involves a few steps. Firstly, the instance that we requested while creating the `Estimator` classes is provisioned and is setup with the appropriate libraries. Then, the data from our channels are downloaded into the instance. Once this is done, the training job begins. The provisioning and data downloading will take some time, depending on the size of the data. Therefore it might be a few minutes before we start getting training logs for our training jobs. The data logs will also print out Accuracy on the validation data for every epoch after training job has executed `min_epochs`. This metric is a proxy for the quality of the algorithm. 

Once the job has finished a "Job complete" message will be printed. The trained model can be found in the S3 bucket that was setup as `output_path` in the estimator.

In [ ]:
bt_model.fit(inputs=data_channels, logs=True)

## Hosting / Inference
Once the training is done, we can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This will allow us to make predictions (or inference) from the model. Note that we don't have to host on the same type of instance that we used to train. Because instance endpoints will be up and running for long, it's advisable to choose a cheaper instance for inference.

In [ ]:
text_classifier = bt_model.deploy(initial_instance_count = 1,instance_type = 'ml.m4.xlarge')

#### Use JSON format for inference
BlazingText supports `application/json` as the content-type for inference. The payload should contain a list of sentences with the key as "**instances**" while being passed to the endpoint.

In [ ]:
#!head test.csv -n 3

all_rows = []
with open('test.csv', mode='r', encoding='utf-8-sig') as csvinfile:
    csv_reader = csv.reader(csvinfile, delimiter=',')
    for row in csv_reader:
        all_rows.append(row)
#all_rows = all_rows[:int(keep*len(all_rows))]
#print(all_rows)
#pool = Pool(processes=multiprocessing.cpu_count())
#transformed_rows = pool.map(transform_instance, all_rows)
#pool.close() 
#pool.join()

#print(all_rows[0][0])
#print(all_rows[0][2])



#sentences = ["Convair was an american aircraft manufacturing company which later expanded into rockets and spacecraft.",
#            "Berwick secondary college is situated in the outer melbourne metropolitan suburb of berwick ."]

sentences = [all_rows[29][2]]
#print(sentences)
print(all_rows[29][0])

# using the same nltk tokenizer that we used during data preparation for training
tokenized_sentences = [' '.join(nltk.word_tokenize(sent)) for sent in sentences]

payload = {"instances" : tokenized_sentences}

response = text_classifier.predict(json.dumps(payload))

predictions = json.loads(response)
print(json.dumps(predictions, indent=2))

By default, the model will return only one prediction, the one with the highest probability. For retrieving the top k predictions, you can set `k` in the configuration as shown below:

In [ ]:
payload = {"instances" : tokenized_sentences,
          "configuration": {"k": 2}}

response = text_classifier.predict(json.dumps(payload))

predictions = json.loads(response)
print(json.dumps(predictions, indent=2))

### Stop / Close the Endpoint (Optional)
Finally, we should delete the endpoint before we close the notebook if we don't need to keep the endpoint running for serving realtime predictions.

In [ ]:
sess.delete_endpoint(text_classifier.endpoint)